In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from pathlib import Path
import os
import sys
from IPython.display import display

In [3]:
def on_kaggle() -> bool:
    try:
        import gresearch_crypto
        return True
    except ModuleNotFoundError:
        return False

In [4]:
# HYPER PARAMETERS
ON_KAGGLE = on_kaggle()
SAMPLE_LEVEL = 1
USE_SAMPLE = SAMPLE_LEVEL == 1
USE_TOY_SAMPLE = SAMPLE_LEVEL == 2

FORCE_REWRITE = True#(ON_KAGGLE and SAMPLE_LEVEL == 0)

In [5]:
if not ON_KAGGLE and os.path.abspath('.').endswith('notebook'):
    os.chdir('../')

In [6]:
#IMPORT_SCRIPT!
if not ON_KAGGLE and '.' not in sys.path:
    sys.path.append('.')
from src.metrics import *
from src.preprocessing import inference, EXPECTED_RAW_COLS
from src.preprocessing.feature_gen import compute_instant_features
from src.preprocessing.ingest_data import merge_asset_details, infer_dtypes
from src.cv import TimeSeriesSplit, get_date_range
from src.modeling import Evaluator
from src.metrics import compute_metrics

In [7]:
if ON_KAGGLE:
    RAW_DIR = Path('../input/g-research-crypto-forecasting/')
    SAMPLE_DIR = Path('../input/create-sample-dataset/data/raw/sample/')
    TOY_SAMPLE_DIR = Path('../input/create-sample-dataset/data/raw/toy_sample/')
else:
    RAW_DIR =  Path('data/raw')
    TOY_SAMPLE_DIR = RAW_DIR.joinpath('toy_sample')
    SAMPLE_DIR = RAW_DIR.joinpath('sample')

# filename
TRAIN_FILE = 'train.csv'
ASSET_DETAILS_PATH = RAW_DIR / 'asset_details.csv'

In [8]:
if USE_TOY_SAMPLE:
    print('USING TOY DATASET')
    RAW_TRAIN_PATH = TOY_SAMPLE_DIR / TRAIN_FILE

elif USE_SAMPLE:
    print('USING SAMPLE DATASET')
    RAW_TRAIN_PATH = SAMPLE_DIR / TRAIN_FILE

else:
    print('USING RAW DATASET')
    RAW_TRAIN_PATH = RAW_DIR / TRAIN_FILE

assert RAW_TRAIN_PATH.exists()

USING SAMPLE DATASET


In [9]:
!ls {RAW_DIR}

asset_details.csv		   sample
example_sample_submission.csv	   supplemental_train.csv
example_test.csv		   toy_sample
g-research-crypto-forecasting.zip  train.csv
gresearch_crypto


In [ ]:
%%time 
raw_data = pd.read_csv(RAW_TRAIN_PATH)

## create train set

In [ ]:
PREPRO_PARAMS = {'window': 60}
MAIN_INDEX = ['timestamp', 'Asset_ID']

In [ ]:
# get valid data only, drop where the target is NaN 
data = raw_data[MAIN_INDEX + ['Target']].dropna(subset=['Target'])
# format time to human readable 
data['time'] = pd.to_datetime(data['timestamp'], unit='s')
# merge asset names
data = merge_asset_details(data, ASSET_DETAILS_PATH)

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
features_df = raw_data.loc[:, EXPECTED_RAW_COLS]
features_df = infer_dtypes(features_df)

In [ ]:
features_df = compute_instant_features(features_df)

In [ ]:
features_df

In [ ]:
assert features_df['timestamp'].isna().sum() == 0

In [ ]:
data = data.merge(features_df, on=MAIN_INDEX, how='left')

In [ ]:
data.isna().mean()

In [ ]:
list(data.columns)

In [ ]:
FEATURES = [
'Count',
'High',
'Low',
'Close',
'Volume',
'VWAP',
'high_low_return',
'open_close_return',
'upper_shadow',
'lower_shadow',
'dolar_amount',
'vol_per_trades'
]

In [ ]:
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, Ridge
from sklearn.preprocessing import MinMaxScaler, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from typing import List


class FilterFeatures(BaseEstimator, TransformerMixin):
    def __init__(self,
                 features: List[str] = None,
                 sort: bool = False):
        self.sort = sort
        self.features = features[:]
        if self.sort:
            self.features.sort()
    
    def fit(self, X: pd.DataFrame, y=None):
        return self
    
    def transform(self, X: pd.DataFrame):
        return X.loc[:, self.features]

    
def test_filter_features():
    n_feat = 100
    size = 500
    t = 100
    expected_columns = [f'{f}_ft' for f in np.arange(n_feat)]
    
    expected_df = pd.DataFrame(np.random.randn(size, n_feat), columns=expected_columns)
    tmf = FilterFeatures(expected_columns)
    tmf.fit(expected_df)
    
    for i in range(t):
        shuffle_columns = np.random.permutation(expected_columns)
        shuffle_df = expected_df.loc[:, shuffle_columns]
        actual_columns = list(tmf.transform(shuffle_df).columns)
        assert (actual_columns == expected_columns), f'cols do not match at iter {i}'
    
test_filter_features()

def build_model(params={'alpha': 0.001}):
    model = Pipeline([('filter', FilterFeatures(FEATURES)),
                      ('norm', MinMaxScaler()),
                      ('model', Ridge(params['alpha']))])
    return model

In [ ]:
def train_model(config, train_data, valid_data, pipeline=None):
    model  = build_model(config['model'])

    if config['training']['time_decay_alpha'] is not None:
        time_decay_alpha = config['training']['time_decay_alpha']
        print(f'using exponential_time_decay with alpha {time_decay_alpha}')
        timesteps = ((train_data['timestamp'].max() - train_data['timestamp'])//60//60//24)
        weight = time_decay_alpha ** timesteps
    else:
        weight = None

    model.fit(train_data[FEATURES], train_data['Target'], model__sample_weight=weight)
    
    return model

In [ ]:
PULIC_LB_RANGE = ['2021-06-13 00:00:00',
                  '2021-09-13 00:00:00'] # 3 MONTH WORTH OF DATA

if USE_TOY_SAMPLE:
    EVAL_PERIODS = [['2021-09-15', '2021-09-22']]

else:
    EVAL_PERIODS = [PULIC_LB_RANGE]

In [ ]:
CV_PARAMS = {'gap_unit': 'min', 'dt_col': 'time'}

CV = TimeSeriesSplit(EVAL_PERIODS, **CV_PARAMS)

In [ ]:
train_idx, valid_idx = next(iter(CV.split(data)))

In [ ]:
train_data = data.loc[train_idx, :].reset_index(drop=True)

In [ ]:
valid_data = data.loc[valid_idx, :].reset_index(drop=True)

In [ ]:
get_date_range(train_data['time'])

In [ ]:
get_date_range(valid_data['time'])

In [ ]:
MODEL_CONFIG = {'alpha': 0.001}
TRAIN_CONFIG = {'time_decay_alpha': 0.99}
CONFIG = {'model': MODEL_CONFIG, 'training': TRAIN_CONFIG}

In [ ]:
MODELS = {}
for asset_id, train_asset_data in data.groupby("Asset_ID"):
    print(f'training model for asset_ID == {asset_id}')
    train_asset_data = train_asset_data.reset_index(drop=True)    
    model = train_model(CONFIG, train_asset_data, train_asset_data)
    MODELS[asset_id] = model

In [ ]:
if not ON_KAGGLE:
    sys.path.append(str(RAW_DIR))

import gresearch_crypto
import traceback
env = gresearch_crypto.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission

In [ ]:
for i, (raw_test_df, submission) in enumerate(iter_test):

    submission = inference(test_data=raw_test_df, submission=submission,
                            models=MODELS)
    if i % 1000 == 0 or i < 10:
        display(submission)    
    env.predict(submission)